In [1]:
print("Guys lets do this")

Guys lets do this


In [5]:
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

In [19]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
users=pd.read_csv('users.csv')

# Display basic info
print("Movies:\n",movies.info())
print("\nRatings:\n",ratings.info())
print("\nUsers:\n",users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3706 entries, 0 to 3705
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie_id      3706 non-null   int64 
 1   movie_title   3706 non-null   object
 2   movie_genres  3706 non-null   object
 3   poster_url    3648 non-null   object
dtypes: int64(1), object(3)
memory usage: 115.9+ KB
Movies:
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   user_id      1000209 non-null  int64  
 1   movie_id     1000209 non-null  int64  
 2   user_rating  1000209 non-null  float64
 3   timestamp    1000209 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 30.5 MB

Ratings:
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 6 columns):
 #   Column                

In [20]:
# Download necessary resources
nltk.download('punkt_tab')

# Drop duplicates and missing values
movies.drop_duplicates(inplace=True)
movies.dropna(inplace=True)

# Split genres into lists
movies['movie_genres'] = movies['movie_genres'].apply(lambda x: x.split('|'))

# One-hot encode genres
mlb = MultiLabelBinarizer()
genre_dummies = pd.DataFrame(mlb.fit_transform(movies['movie_genres']), columns=mlb.classes_)
movies = pd.concat([movies.drop('movie_genres', axis=1), genre_dummies], axis=1)
movies['movie_title'] = movies['movie_title'].astype(str)
# Clean and tokenize titles
def clean_title(title):
    return re.sub(r'\(\d{4}\)', '', title).strip()

movies['clean_title']=movies['movie_title'].apply(clean_title)
movies['title_tokens']=movies['clean_title'].apply(word_tokenize)
movies['title_token_count']=movies['title_tokens'].apply(len)
print(movies.head())

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


   movie_id                   movie_title  \
0    3107.0              Backdraft (1991)   
1    2114.0         Outsiders, The (1983)   
2     256.0                 Junior (1994)   
3    1389.0               Jaws 3-D (1983)   
4    3635.0  Spy Who Loved Me, The (1977)   

                                          poster_url    0  0, 1  0, 1, 10  \
0  https://image.tmdb.org/t/p/w500/bymJtYNIdiAiBR...  0.0   0.0       0.0   
1  https://image.tmdb.org/t/p/w500/pl8Tf36TAOb2i5...  0.0   0.0       0.0   
2  https://image.tmdb.org/t/p/w500/4tRF43CbEYHmV8...  0.0   0.0       0.0   
3  https://image.tmdb.org/t/p/w500/kqDXj53F9paqVG...  0.0   0.0       0.0   
4  https://image.tmdb.org/t/p/w500/3ZxHKFxMYvAko6...  1.0   0.0       0.0   

   0, 1, 10, 16  0, 1, 13  0, 1, 13, 15  0, 1, 14  ...  9, 10  9, 13  \
0           0.0       0.0           0.0       0.0  ...    0.0    0.0   
1           0.0       0.0           0.0       0.0  ...    0.0    0.0   
2           0.0       0.0           0.0       0.0 

In [14]:
# Drop duplicates and missing values
ratings.drop_duplicates(inplace=True)
ratings.dropna(inplace=True)

# Convert timestamp to datetime
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Extract date features
ratings['year'] = ratings['timestamp'].dt.year
ratings['month'] = ratings['timestamp'].dt.month
ratings['day'] = ratings['timestamp'].dt.day
print(ratings.head())

   user_id  movie_id  user_rating           timestamp  year  month  day
0      130      3107          5.0 2000-12-21 20:56:33  2000     12   21
1     3829      2114          4.0 2000-08-10 18:42:47  2000      8   10
2     1265       256          1.0 2002-01-27 03:52:32  2002      1   27
3     2896      1389          5.0 2000-10-20 01:16:45  2000     10   20
4     5264      3635          4.0 2000-06-16 18:28:31  2000      6   16


In [16]:
# Drop duplicates and missing values
users.drop_duplicates(inplace=True)
users.dropna(inplace=True)

# Convert to categorical
users['gender'] = users['user_gender'].astype('category')
users['occupation'] = users['user_occupation_label'].astype('category')

# Encode gender and occupation
users['gender_encoded'] = LabelEncoder().fit_transform(users['gender'])
users['occupation_encoded'] = LabelEncoder().fit_transform(users['occupation'])

print(users.head())

   user_id  user_gender  bucketized_user_age  user_occupation_label  \
0      130         True                 35.0                     18   
1     3829        False                 25.0                      0   
2     1265        False                 18.0                     21   
3     2896         True                 18.0                     14   
4     5264         True                 18.0                     17   

   user_occupation_text user_zip_code gender occupation  gender_encoded  \
0   technician/engineer         50021   True         18               1   
1     academic/educator         22307  False          0               0   
2                writer         49321  False         21               0   
3       sales/marketing         60073   True         14               1   
4  college/grad student         15217   True         17               1   

   occupation_encoded  
0                  16  
1                   0  
2                  19  
3                  12  
4 

In [ ]:
# Merge all into one DataFrame
df = ratings.merge(users, on='user_id').merge(movies, on='movie_id')
print(df.head())